In [2]:
!pip install gradio langchain openai pydantic


In [3]:
!pip install gradio langchain openai pydantic tiktoken tenacity aiohttp


In [ ]:
with open(".env", "w") as f:
    f.write("\n")

In [5]:
# ✅ 환경 변수 로딩
load_dotenv("/workspace/.env")  # .env에 OPENAI_API_KEY 저장 필요

NameError: name 'load_dotenv' is not defined

In [6]:
pip install faiss-cpu langchain openai pypdf


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gradio as gr
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import StructuredTool
from pydantic import BaseModel
import os
import re

# OpenAI 키 설정
os.environ["OPENAI_API_KEY"] = "" 


# 숫자 단위 전처리
def convert_korean_numbers(text: str) -> str:
    units = {"천": 1_000, "만": 10_000, "억": 100_000_000}
    pattern = re.compile(r'(\d+)(천|만|억)')
    def repl(match):
        return str(int(match.group(1)) * units[match.group(2)])
    return pattern.sub(repl, text)

# 전세대출 한도 계산기
def calculate_jeonse_limit(deposit: int, income: int, credit_grade: str) -> str:
    ratio = 0.8 if credit_grade == "상" else 0.6 if credit_grade == "중" else 0.4
    limit = int(min(deposit * ratio, income * 4))
    return f"당신의 전세대출 한도는 약 {limit:,}원입니다."

# 월 상환금 계산기
def calculate_monthly_payment(loan_amount: int, annual_interest_rate: float, years: int) -> str:
    r = annual_interest_rate / 100 / 12
    n = years * 12
    monthly = loan_amount * r / (1 - (1 + r) ** -n)
    return f"{loan_amount:,}원을 연 {annual_interest_rate}%의 이자로 {years}년간 갚을 경우, 매월 약 {int(monthly):,}원을 상환하셔야 합니다."

# 스키마 정의
class JeonseLimitInput(BaseModel):
    deposit: int
    income: int
    credit_grade: str

class MonthlyPaymentInput(BaseModel):
    loan_amount: int
    annual_interest_rate: float
    years: int

# 도구 등록
tools = [
    StructuredTool.from_function(
        name="jeonse_limit_calc",
        description="전세대출 한도 계산기",
        args_schema=JeonseLimitInput,
        func=calculate_jeonse_limit
    ),
    StructuredTool.from_function(
        name="monthly_payment_calc",
        description="월 상환금 계산기",
        args_schema=MonthlyPaymentInput,
        func=calculate_monthly_payment
    )
]

# LLM 및 메모리 구성
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 금융 상담 전문가입니다."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

# 챗봇 함수
chat_history = []
def chatbot(user_input):
    global chat_history
    try:
        processed_input = convert_korean_numbers(user_input)
        response = agent_executor.invoke({"input": processed_input})

        # 핵심 응답만 추출
        answer = response["output"]

        # 히스토리 저장
        chat_history.append(("👤 사용자", user_input))
        chat_history.append(("🤖 챗봇", answer))

        # 포맷팅된 히스토리 출력
        messages = "\n\n".join([f"{role}:\n{msg}" for role, msg in chat_history])
        return messages

    except Exception as e:
        return f"❗ 오류 발생: {str(e)}"


# Gradio UI 실행 (외부 공유 링크 자동 생성)
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="질문을 입력하세요"),
    outputs=gr.Textbox(label="챗봇 응답"),
    title="💬 금융 상담 챗봇"
)

iface.launch(share=True)  # 🔗 공유 링크 자동 생성됨



/tmp/ipykernel_4629/1606211725.py:64: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0)
/tmp/ipykernel_4629/1606211725.py:65: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://96be161dbb3bde3e61.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import os
import re
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.messages import SystemMessage
from pydantic import BaseModel

# 🔐 OpenAI API Key 설정
os.environ["OPENAI_API_KEY"] = ""  # ← 본인의 키로 교체

# ✅ 숫자 단위 전처리
def convert_korean_numbers(text: str) -> str:
    units = {"천": 1_000, "만": 10_000, "억": 100_000_000}
    pattern = re.compile(r'(\d+)(천|만|억)')
    def repl(match):
        return str(int(match.group(1)) * units[match.group(2)])
    return pattern.sub(repl, text)

# ✅ 계산 함수들
def calculate_jeonse_limit(deposit: int, income: int, credit_grade: str) -> str:
    ratio = 0.8 if credit_grade == "상" else 0.6 if credit_grade == "중" else 0.4
    limit = int(min(deposit * ratio, income * 4))
    return f"당신의 전세대출 한도는 약 {limit:,}원입니다."

def calculate_monthly_payment(loan_amount: int, annual_interest_rate: float, years: int) -> str:
    r = annual_interest_rate / 100 / 12
    n = years * 12
    monthly = loan_amount * r / (1 - (1 + r) ** -n)
    return f"{loan_amount:,}원을 연 {annual_interest_rate}%의 이자로 {years}년간 갚을 경우, 매월 약 {int(monthly):,}원을 상환하셔야 합니다."

# ✅ Pydantic 스키마
class JeonseLimitInput(BaseModel):
    deposit: int
    income: int
    credit_grade: str

class MonthlyPaymentInput(BaseModel):
    loan_amount: int
    annual_interest_rate: float
    years: int

# ✅ 도구 정의
tools = [
    StructuredTool.from_function(
        name="jeonse_limit_calc",
        description="전세대출 한도 계산기",
        args_schema=JeonseLimitInput,
        func=calculate_jeonse_limit
    ),
    StructuredTool.from_function(
        name="monthly_payment_calc",
        description="월 상환금 계산기",
        args_schema=MonthlyPaymentInput,
        func=calculate_monthly_payment
    )
]

# ✅ 프롬프트 구성 (ChatPromptTemplate 기반)
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""
당신은 은행 대출 전문가 AI입니다.  
사용자에게 친절하고 상세하게 대출 상품을 안내하며,  
전세자금대출과 주택담보대출에 관한 정확하고 신뢰성 있는 정보를 제공합니다.  
사용자의 질문과 관련된 문서 내용을 바탕으로 구체적인 답변을 작성하세요.  
계산이 필요한 경우 공식과 필요한 값을 요청하고 계산 후 결과를 제안하세요.
"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# ✅ LLM 및 AgentExecutor 설정
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

# ✅ 챗봇 함수 정의
chat_history = []

def chatbot(user_input):
    global chat_history
    try:
        processed_input = convert_korean_numbers(user_input)
        response = agent_executor.invoke({"input": processed_input})
        answer = response["output"]

        chat_history.append(("👤 사용자", user_input))
        chat_history.append(("🤖 챗봇", answer))

        return "\n\n".join([f"{role}:\n{msg}" for role, msg in chat_history])
    except Exception as e:
        return f"❗ 오류 발생: {str(e)}"

# ✅ Gradio UI 실행
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="질문을 입력하세요", lines=2),
    outputs=gr.Textbox(label="챗봇 응답", lines=10),
    title="💬 금융 상담 챗봇"
)

iface.launch(share=True)


In [ ]:
import os
import json
import re
import numpy as np
import faiss
import gradio as gr
from pydantic import BaseModel
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.messages import SystemMessage
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 🔐 OpenAI API Key 설정
os.environ["OPENAI_API_KEY"] = "
# ✅ 숫자 단위 전처리
def convert_korean_numbers(text: str) -> str:
    units = {"천": 1_000, "만": 10_000, "억": 100_000_000}
    pattern = re.compile(r'(\d+)(천|만|억)')
    def repl(match): return str(int(match.group(1)) * units[match.group(2)])
    return pattern.sub(repl, text)

# ✅ 계산 함수 정의
def calculate_jeonse_limit(deposit: int, income: int, credit_grade: str) -> str:
    ratio = 0.8 if credit_grade == "상" else 0.6 if credit_grade == "중" else 0.4
    limit = int(min(deposit * ratio, income * 4))
    return f"전세대출 한도는 약 {limit:,}원입니다."

def calculate_monthly_payment(loan_amount: int, annual_interest_rate: float, years: int) -> str:
    r = annual_interest_rate / 100 / 12
    n = years * 12
    monthly = loan_amount * r / (1 - (1 + r) ** -n)
    return f"{loan_amount:,}원을 연 {annual_interest_rate}%로 {years}년간 갚을 경우, 월 {int(monthly):,}원 상환입니다."

# ✅ Tool 입력 스키마
class JeonseLimitInput(BaseModel):
    deposit: int
    income: int
    credit_grade: str

class MonthlyPaymentInput(BaseModel):
    loan_amount: int
    annual_interest_rate: float
    years: int

# ✅ 계산 도구 정의
tools = [
    StructuredTool.from_function(
        name="jeonse_limit_calc",
        description="전세대출 한도 계산기",
        args_schema=JeonseLimitInput,
        func=calculate_jeonse_limit
    ),
    StructuredTool.from_function(
        name="monthly_payment_calc",
        description="월 상환금 계산기",
        args_schema=MonthlyPaymentInput,
        func=calculate_monthly_payment
    )
]

# ✅ JSON 문서 로딩 및 벡터 임베딩 함수 (에러 방지 포함)
def load_and_embed_cleaned_json(json_path):
    import json
    import numpy as np
    import faiss
    from langchain.schema import Document
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.vectorstores.faiss import FAISS
    from langchain.embeddings import OpenAIEmbeddings

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    documents = [
        Document(page_content=item["content"], metadata={"source": item["source"]})
        for item in data
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = splitter.split_documents(documents)
    docs = [doc for doc in docs if len(doc.page_content.strip()) > 30]

    print(f"🔍 문서 개수: {len(docs)}")
    for i, d in enumerate(docs[:3]):
        print(f"[문서 {i+1} 길이] {len(d.page_content)}자")
        print(f"[내용 요약] {d.page_content[:100]}...\n")

    if not docs:
        raise ValueError("❌ 유효한 문서가 없습니다.")

    # ✅ 텍스트 및 메타데이터 준비
    texts = [doc.page_content for doc in docs]
    metadatas = [doc.metadata for doc in docs]

    # ✅ 임베딩 모델
    embeddings_model = OpenAIEmbeddings()

    # ✅ FAISS 벡터스토어 생성 (LangChain 제공 공식 방식)
    vectorstore = FAISS.from_texts(texts=texts, embedding=embeddings_model, metadatas=metadatas)

    return vectorstore.as_retriever()

# ✅ RAG 벡터 검색기 생성
retriever = load_and_embed_cleaned_json("/workspace/cleaned_data.json")

# ✅ 프롬프트 구성
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="""
당신은 은행 대출 전문가 AI입니다.  
아래 문서를 참고하여 사용자 질문에 구체적이고 신뢰성 있게 답하십시오.  
계산이 필요한 경우, 필요한 값을 요청하고 계산 결과를 알려주세요.

### 문서:
{context}
"""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ],
    input_variables=["context", "input", "agent_scratchpad"]
)
# ✅ 에이전트 및 메모리 구성
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

# ✅ 챗봇 처리 함수
chat_history = []

def chatbot(user_input):
    global chat_history
    try:
        processed_input = convert_korean_numbers(user_input)
        docs = retriever.get_relevant_documents(processed_input)
        context = "\n\n".join([doc.page_content for doc in docs[:3]])

        response = agent_executor.invoke({
            "input": processed_input,
            "context": context
        })

        answer = response["output"]
        chat_history.append(("👤 사용자", user_input))
        chat_history.append(("🤖 챗봇", answer))

        return "\n\n".join([f"{role}:\n{msg}" for role, msg in chat_history])
    except Exception as e:
        return f"❗ 오류 발생: {str(e)}"

# ✅ Gradio UI 실행
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="질문을 입력하세요", lines=2),
    outputs=gr.Textbox(label="챗봇 응답", lines=10),
    title="📄 문서 기반 금융 상담 챗봇"
)

iface.launch(share=True)


🔍 문서 개수: 1689
[문서 1 길이] 72자
[내용 요약] 관심상품추가 전세대출 영업점
고정금리 전세자금대출
임차보증금의 90%이내, 최고 4억원까지 이용할 수 있는 고정금리 전세대출 상품...

[문서 2 길이] 455자
[내용 요약] 상품특징
한국주택금융공사의 보증서를 담보로 임차보증금의 90%내에서 주택의 전세자금을 고정금리로 지원해드리는 상품입니다.
대출대상
주택임대차계약을 체결한 국민인 거주자로 아래의 조...

[문서 3 길이] 486자
[내용 요약] 대출금리
[기준일자: 2024.01.05. 단위:연%]
금리구분 기준금리 가산금리 우대금리 최고-최저금리
금융채 2년물 유통수익률 3.661 1.000 0.000 4.661
- 2...



TypeError: ChatPromptTemplate.from_messages() got an unexpected keyword argument 'input_variables'

In [ ]:
import os
import json
import re
import numpy as np
import faiss
import gradio as gr
from pydantic import BaseModel
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.messages import SystemMessage
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 🔐 OpenAI API Key 설정
os.environ["OPENAI_API_KEY"] = ""

# ✅ 숫자 단위 전처리
def convert_korean_numbers(text: str) -> str:
    units = {"천": 1_000, "만": 10_000, "억": 100_000_000}
    pattern = re.compile(r'(\d+)(천|만|억)')
    def repl(match): return str(int(match.group(1)) * units[match.group(2)])
    return pattern.sub(repl, text)

# ✅ 계산 함수 정의
def calculate_jeonse_limit(deposit: int, income: int, credit_grade: str) -> str:
    ratio = 0.8 if credit_grade == "상" else 0.6 if credit_grade == "중" else 0.4
    limit = int(min(deposit * ratio, income * 4))
    return f"전세대출 한도는 약 {limit:,}원입니다."

def calculate_monthly_payment(loan_amount: int, annual_interest_rate: float, years: int) -> str:
    r = annual_interest_rate / 100 / 12
    n = years * 12
    monthly = loan_amount * r / (1 - (1 + r) ** -n)
    return f"{loan_amount:,}원을 연 {annual_interest_rate}%로 {years}년간 갚을 경우, 월 {int(monthly):,}원 상환입니다."

# ✅ Tool 입력 스키마
class JeonseLimitInput(BaseModel):
    deposit: int
    income: int
    credit_grade: str

class MonthlyPaymentInput(BaseModel):
    loan_amount: int
    annual_interest_rate: float
    years: int

# ✅ 계산 도구 정의
tools = [
    StructuredTool.from_function(
        name="jeonse_limit_calc",
        description="전세대출 한도 계산기",
        args_schema=JeonseLimitInput,
        func=calculate_jeonse_limit
    ),
    StructuredTool.from_function(
        name="monthly_payment_calc",
        description="월 상환금 계산기",
        args_schema=MonthlyPaymentInput,
        func=calculate_monthly_payment
    )
]

# ✅ JSON 문서 로딩 및 벡터 임베딩 함수
def load_and_embed_cleaned_json(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    documents = [
        Document(page_content=item["content"], metadata={"source": item["source"]})
        for item in data
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = splitter.split_documents(documents)
    docs = [doc for doc in docs if len(doc.page_content.strip()) > 30]

    print(f"\U0001f50d 문서 개수: {len(docs)}")
    for i, d in enumerate(docs[:3]):
        print(f"[문서 {i+1} 길이] {len(d.page_content)}자")
        print(f"[내용 요약] {d.page_content[:100]}...\n")

    if not docs:
        raise ValueError("❌ 유효한 문서가 없습니다.")

    texts = [doc.page_content for doc in docs]
    metadatas = [doc.metadata for doc in docs]
    embeddings_model = OpenAIEmbeddings()

    vectorstore = FAISS.from_texts(texts=texts, embedding=embeddings_model, metadatas=metadatas)
    return vectorstore.as_retriever()

# ✅ RAG 벡터 검색기 생성
retriever = load_and_embed_cleaned_json("/workspace/cleaned_data.json")

# ✅ 프롬프트 구성
prompt = ChatPromptTemplate(
    messages=[
        SystemMessage(content="""
당신은 은행 대출 전문가 AI입니다.  
사용자 질문에 전문적이고 친절하게 응답하세요.
"""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# ✅ 에이전트 및 메모리 구성
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

# ✅ 챗봇 처리 함수
chat_history = []

def chatbot(user_input):
    global chat_history
    try:
        processed_input = convert_korean_numbers(user_input)
        docs = retriever.get_relevant_documents(processed_input)
        context = "\n\n".join([doc.page_content for doc in docs[:3]])

        # context를 input에 병합
        combined_input = f"""
[참고 문서 요약]
{context}

[사용자 질문]
{processed_input}
"""

        response = agent_executor.invoke({"input": combined_input})
        answer = response["output"]
        chat_history.append(("\U0001f464 사용자", user_input))
        chat_history.append(("\U0001f916 챗봇", answer))

        return "\n\n".join([f"{role}:\n{msg}" for role, msg in chat_history])
    except Exception as e:
        return f"❗ 오류 발생: {str(e)}"

# ✅ Gradio UI 실행
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="질문을 입력하세요", lines=2),
    outputs=gr.Textbox(label="챗봇 응답", lines=10),
    title="📄 문서 기반 금융 상담 챗봇"
)

iface.launch(share=True)

🔍 문서 개수: 1689
[문서 1 길이] 72자
[내용 요약] 관심상품추가 전세대출 영업점
고정금리 전세자금대출
임차보증금의 90%이내, 최고 4억원까지 이용할 수 있는 고정금리 전세대출 상품...

[문서 2 길이] 455자
[내용 요약] 상품특징
한국주택금융공사의 보증서를 담보로 임차보증금의 90%내에서 주택의 전세자금을 고정금리로 지원해드리는 상품입니다.
대출대상
주택임대차계약을 체결한 국민인 거주자로 아래의 조...

[문서 3 길이] 486자
[내용 요약] 대출금리
[기준일자: 2024.01.05. 단위:연%]
금리구분 기준금리 가산금리 우대금리 최고-최저금리
금융채 2년물 유통수익률 3.661 1.000 0.000 4.661
- 2...

* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://b13e5e2a1f9040588f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...

Invoking: `jeonse_limit_calc` with `{'deposit': 200000000, 'income': 50000000, 'credit_grade': '2등급'}`


전세대출 한도는 약 80,000,000원입니다.안녕하세요, 고객님. 전문적인 대출 상담을 위해 연봉, 신용등급, 희망 대출 금액, 상환기간 등을 고려해보았습니다. 

고객님의 경우, 연봉이 5000만원이고 신용등급이 2등급이라는 점을 고려하면 전세대출 한도는 약 8천만원으로 계산됩니다. 

하지만 고객님께서 희망하시는 대출 금액이 2억원이고, 상환기간이 20년이라는 점을 고려하면, 현재의 상황에서는 고객님의 희망에 부합하는 대출 상품을 찾기 어려울 수 있습니다. 

대출 상품은 고객님의 신용등급, 연봉, 부채 등 여러 요인을 고려하여 결정되므로, 상황에 따라 변동이 있을 수 있습니다. 따라서, 좀 더 자세한 상담을 원하신다면 직접 은행에 방문하시거나 전화 상담을 이용해보시는 것을 권장드립니다. 

항상 고객님의 금융 건강을 위해 최선을 다하겠습니다. 감사합니다.

> Finished chain.


> Entering new AgentExecutor chain...
KB국민은행에서는 다양한 대출 상품을 제공하고 있습니다. 

1. KB Star 신용대출: 신용등급과 소득을 기반으로 한 대출 상품으로, 대출 한도와 금리는 개인의 신용 상태에 따라 달라집니다.

2. KB Star 아파트담보대출: 아파트를 담보로 설정하여 대출을 받을 수 있는 상품입니다. 담보 가치와 소득, 신용 상태 등을 종합적으로 고려하여 대출 한도와 금리가 결정됩니다.

3. KB Star 전세자금대출: 전세 계약금을 대출 받을 수 있는 상품으로, 전세 계약금의 최대 80%까지 대출 받을 수 있습니다.

4. KB Star 주택담보대출: 주택을 담보로 설정하여 대출을 받을 수 있는 상품입니다. 담보 가치와 소득, 신용 상태 등을 종합적